## Generate a data sample for SVDTime Neural Network training

This script generates a toy data sample for neural network training.
The result is stored in a ROOT file. 

TO DO / FIX: 
* We can make the generation faster using C++. 
* root_pandas doesn't work. HDF5 works perfectly, but requires complicated installation on linux. So we stay with pickle (*.pkl) for the time being.

Packages required:
- pandas
- SVDSiimBase

In [1]:
import pandas as pd
from SVDSimBase import *

### Sample generation

Generate a pandas dataframe containing a large number of waveform samples and truth data, and store it in HDF5.
The data will be used as training and test data.
Waveform widths (tau), amplitudes and time shifts are sampled uniformly from a large set of feasible values.

In [2]:
n_samples = 1000000
generator = SampleGenerator(
    (-2.5*dt, 1.5*dt), 
    (tau_hao2real(raw_tau_min), tau_hao2real(raw_tau_max)), 
    (3,100), 
    (1,4),
    3)
print('Generating {0} samples...'.format(n_samples))
sample = generator.generate(n_samples)

# Create a bin table
timearray = generator.get_t0_array()
timebins = generator.get_t0_bins()
bins = pd.DataFrame({
    'midpoint' : timearray,
    'lower' : timebins.values[:-1],
    'upper' : timebins.values[1:]
})

# Create a table of simulation bounds
bounds = pd.DataFrame({
    'value': np.array(['t0', 'amplitude', 'tau', 'sigma']),
    'sampling': np.array(['uniform', 'uniform', 'uniform', 'uniform']),
    'low'  : [
        generator.get_t0_bounds()[0], 
        generator.get_amp_bounds()[0], 
        tau_hao2real(raw_tau_min),
        generator.get_sigma_bounds()[0]
    ],
    'high' : [
        generator.get_t0_bounds()[1], 
        generator.get_amp_bounds()[1], 
        tau_hao2real(raw_tau_max),
        generator.get_sigma_bounds()[1]
    ]
})
orderedcols = ['value', 'sampling', 'low', 'high']
bounds = bounds[orderedcols]

print('Samples created.')

Generating 1000000 samples...
Samples created.


In [3]:
output_name = 'SVDTime_Training{0}_{1}.pkl'

# There will be three trees: sample, bins, bounds.

sample.to_pickle(output_name.format('Sample', n_samples))
bins.to_pickle(output_name.format('Bins', n_samples))
bounds.to_pickle(output_name.format('Bounds', n_samples))

print('Done.')

Done.


In [4]:
sample.head()

,test,amplitude,t0,tau,sigma,s1,s2,s3,s4,s5,s6,normed_tau,t0_bin,abin
0,0.420250,45.085136,0.486506,271.287924,1.764844,0.000000,0.000000,30.597607,44.763166,42.496677,32.864097,44.632101,16,43
1,0.470006,40.923811,-76.319618,327.200027,1.863665,32.731206,40.779863,37.023823,32.194629,24.682549,18.780200,83.199794,1,38
2,0.752444,47.280162,18.665801,304.865337,2.584116,0.000000,0.000000,10.061467,38.310970,46.824519,43.728683,67.793515,20,45
3,0.700490,9.206401,-33.538889,230.937656,1.520102,0.000000,7.236357,8.552058,8.552058,6.578506,3.947104,16.798830,9,7
4,0.892159,54.405736,36.581858,344.323744,3.623510,0.000000,0.000000,0.000000,23.733895,48.571691,53.815226,95.011588,23,52
